# MeowLab

> cat可能是你最常用的命令行工具之一，但是可能很少有人会关注cat的性能问题。
> 
> 但实际上，cat可能比你想的更加高效。在这个lab中，我们会一起来探索cat如此高效的秘密。

**李玉玺 2023202310**

## 什么是cat

对于大部分同学来说，这个问题有些太trivial了。但是以防万一，还是在这里介绍一下，cat是一个GNU coreutils中的一个程序，它的作用是连接(con**cat**enate)一系列文件，并将其输出到标准输出流(stdout)中。

> 如果你愿意去找，你会发现这个lab中所有的代码都可以在GNU coreutils中找到，或者你用某个大模型，它可能也能写得不错。
> 但是，除了写代码以外，这个lab中也会包含一些分析和讨论。因此即使你选择直接去抄GNU coreutils中的代码，也不要直接不加思考的复制粘贴。

## 环境要求

* 操作系统：Linux（任何包含GNU coreutils的发行版）
* 编译器：本lab允许使用C/C++或者Rust，选择你喜欢的任何编译器即可。
* Python3.x：本lab一些地方需要你画图，推荐你使用matplotlib。另外，我们使用一个简单的脚本来生成测试文件。

## 在开始之前

这个项目的所有代码需要你自己动手来写，我们只提供了一个用于生成测试文件的脚本。

为了展示比较明显的测试效果，我们会生成一个2GB的测试文件，请确保你的磁盘拥有足够的空间。你可以运行下面这个单元格来查看你的电脑是否有足够的空间。这里我们使用了`df`命令，这个命令的作用是查看某个文件系统所在的磁盘的使用量。

In [2]:
%%bash
df -h /

avail=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[0-9.]*')
unit=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[a-zA-Z]*')
if [[ "$unit" == "M" || "$unit" == "K" ]]; then
    echo "Disk space is low: $avail$unit"
elif [[ "$unit" == "T" ]]; then
    echo "Disk space is sufficient: $avail$unit"
elif [[ "$unit" == "G" ]]; then
    if (( $(echo "$avail < 10" | bc -l) )); then
        echo "Disk space is low: $avail$unit"
    else
        echo "Disk space is sufficient: $avail$unit"
    fi
else
    echo "Unknown unit: $unit"
fi


Filesystem      Size  Used Avail Use% Mounted on
/dev/sdc       1007G   36G  920G   4% /


Disk space is sufficient: 920G


你可以使用我们提供的python脚本来生成测试文件，运行下面的单元格。测试文件的生成可能会花费一定的时间。

In [2]:
import random

MB = 1024 * 1024

# A static seed for reproducibility
random.seed(42)

with open("test.txt", "wb") as f:
    for _ in range(2048):
        f.write(random.randbytes(1 * MB)) # 1MB of random data

当前文件夹下面会出现一个大小为2GB的文件`test.txt`。你可以运行下面的单元格来验证。

In [9]:
%%bash
ls -lh test.txt

-rw-r--r-- 1 root root 2.0G Jun 10 17:49 test.txt


最后，我们的lab会需要使用`hyperfine`来测量程序的运行时间。我们推荐你使用`cargo`进行安装。你可以从[Rust官网](https://www.rust-lang.org/zh-CN/learn/get-started)找到安装脚本。如果你的电脑已经安装好了`cargo`，你可以运行下面的单元格来安装`hyperfine`。

In [6]:
%%bash
export PATH="/root/.cargo/bin:$PATH"
cargo install hyperfine

    Updating crates.io index
  Downloaded hyperfine v1.19.0
  Installing hyperfine v1.19.0
    Updating crates.io index
     Locking 137 packages to latest compatible versions
      Adding colored v2.2.0 (available: v3.0.0)
      Adding indicatif v0.17.4 (available: v0.17.11)
      Adding nix v0.29.0 (available: v0.30.1)
      Adding rand v0.8.5 (available: v0.9.1)
  Downloaded cfg_aliases v0.2.1
  Downloaded bytecheck_derive v0.6.12
  Downloaded anstyle v1.0.11
  Downloaded autocfg v0.1.8
  Downloaded number_prefix v0.4.0
  Downloaded ptr_meta v0.1.4
  Downloaded ptr_meta_derive v0.1.4
  Downloaded is_terminal_polyfill v1.70.1
  Downloaded equivalent v1.0.2
  Downloaded statistical v1.0.0
  Downloaded tinyvec_macros v0.1.1
  Downloaded bytecheck v0.6.12
  Downloaded funty v2.0.0
  Downloaded errno v0.3.12
  Downloaded cfg-if v1.0.1
  Downloaded ppv-lite86 v0.2.21
  Downloaded num v0.2.1
  Downloaded colorchoice v1.0.4
  Downloaded autocfg v1.4.0
  Downloaded anstyle-query v1.1.3
  Dow

有了以上的准备工作，我们终于可以开始我们的探索之旅了。

## 任务0: 测量cat的效率

你要做的第一个任务十分简单。学习一下如何使用`hyperfine`，并且使用它来测试GNU coreutils中的cat输出我们生成的测试文件`test.txt`的时间。运行下面的单元格来运行测试。

In [13]:
%%bash
export PATH="/root/.cargo/bin:$PATH"
hyperfine --warmup 3 'cat test.txt'

Benchmark 1: cat test.txt
  Time (mean ± σ):     455.7 ms ±  37.8 ms    [User: 5.1 ms, System: 451.0 ms]
  Range (min … max):   414.4 ms … 526.0 ms    10 runs
 


## 任务1: 写一个最朴素的cat

我们现在使用`hyperfine`测量了GNU coreutils中的`cat`的运行效率，但是对于这个运行时间，我们可能并没有什么概念。现在我们将会写一个最朴素的`cat`程序来作为baseline。这个程序需要满足一下要求：
1. 把你的源代码命名为`mycat1.c`或者`mycat1.rs`，并且可执行的二进制文件命名为`mycat1`。
2. 你的程序接受且只接受一个命令行参数，这个参数是你要读取的文件。
3. 你的程序直接使用`read`和`write`系统调用，每次读取并输出一个字符，不使用任何缓冲区。
4. 使用`hpyerfine`测量`mycat1`和`cat`运行时间（`mycat1`的运行时间可能会非常长）
5. 虽然这是一个很简单的程序，但是请仍然保持系统编程的规范。请在你的代码中进行进行所有必要的错误处理。

这个任务不需要在文档中包含任何内容。

In [ ]:
%%bash
hyperfine --warmup 3 --runs 1 './target/mycat1 test.txt'

## 任务2: 带有缓冲区的cat

如果你正确完成了上面的任务，你会发现，`mycat1`的性能和系统中的`cat`的性能相差甚远。但是经过了ics课程的学习，你肯定已经想到了至少一个解决方案——使用缓冲区。缓冲区可以有效减少反复进行系统调用的性能消耗。但是，我们要面临的第二个问题是：缓冲区的大小应该如何设置呢？我们接下来将一步一步地探索这个问题。

我们之前肯定也已经讲过不少涉及IO缓冲区的程序了，它们的缓冲区大小都被设置为多少呢？我相信1024, 4096, 8192一定是比较常见的几个数字。经过ics的学习，我相信你们肯定能说出原因了。那么，这个任务，我们将根据这个原理优化我们的`mycat1`。你至少需要完成下面要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat2.c`/`mycat2.rs`，可执行二进制文件命名为`mycat2`。
2. 写一个`io_blocksize`函数用于确定你的缓冲区的大小，在这个任务中，你可以将缓冲区设置成你当前系统中一个内存页的大小。（注意：你不能假设所有系统中内存页的大小都是4K，请你使用一个系统调用或者标准库中的函数来获取内存页的大小，而不要使用一个固定值。不过允许你使用一个固定值，如果获取内存页发生错误，但是这不应该是一个常见的情况）。
3. 使用标准库提供的函数动态分配用于缓冲区的内存。
4. 使用`hpyerfine`测量`mycat2`的运行时间
5. 请保持系统编程的基本规范。

这个任务不需要在文档中包含任何内容。

In [33]:
%%bash
export PATH="/root/.cargo/bin:$PATH"
hyperfine --warmup 3 './mycat2 test.txt'

Benchmark 1: ./mycat2 test.txt
  Time (mean ± σ):     779.1 ms ±   3.5 ms    [User: 102.6 ms, System: 676.5 ms]
  Range (min … max):   773.4 ms … 784.4 ms    10 runs
 


## 任务3: 缓冲区对齐的cat

如果你正确完成了上面这个任务，你会发现，添加了缓冲区的`mycat2`性能提升十分显著。但是我们还可以进一步优化。实际上只是添加了缓冲区并且设置缓冲区的大小为内存页的整数倍并不是没有什么太多的意义，这样的设置只是为了这个一个任务做铺垫的。在这个任务中，我们将尝试将我们的缓冲区对齐到系统的内存页。至于为什么要这么做，请大家在自己的文档中分析一下。你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat3.c`/`mycat3.rs`，可执行二进制文件命名为`mycat3`。
2. 写两个函数`char* align_alloc(size_t size)`和`void align_free(void* ptr)`，它们的作用分别是分配一段内存，长度不小于`size`并且返回一个对齐到内存页起始的指针`ptr`，以及给出一个先前从`align_alloc`返回的指针并释放之前分配的内存。
3. 利用这两个函数修改你的代码，缓冲区的大小仍然设置成一个内存页的大小。
4. 使用`hpyerfine`测量`mycat3`的运行时间
5. 请保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 为什么将缓冲区对齐到系统的内存可能提高性能？你的实验结果支持这个猜想吗？为什么？
2. 为什么我们直接使用`malloc`函数分配的内存不能对齐到内存页，即使我们分配的内存大小已经是内存页大小的整数倍了。
3. 你是怎么在不知道原始的malloc返回的指针的情况下正确释放内存的？

In [34]:
%%bash
export PATH="/root/.cargo/bin:$PATH"
hyperfine --warmup 3 './mycat3 test.txt'

Benchmark 1: ./mycat3 test.txt
  Time (mean ± σ):     775.7 ms ±   4.7 ms    [User: 90.5 ms, System: 685.3 ms]
  Range (min … max):   766.6 ms … 780.7 ms    10 runs
 


1.
**CPU缓存效率**：现代CPU的缓存行通常是64字节或128字节，而内存页通常是4KB。如果缓冲区跨越页边界或者没有对齐，内核可能需要进行额外的内存操作。页对齐的内存访问可以避免跨越缓存行边界，减少缓存缺失。  
**TLB（Translation Lookaside Buffer）效率**：TLB用于缓存虚拟地址到物理地址的映射。页对齐的内存访问可以更好地利用TLB，减少地址转换开销。 
**操作系统优化**：许多操作系统内核函数（如read和write）对页对齐的缓冲区有特殊优化路径，可能避免额外的内存拷贝。  
**我的实验结果大概支持了这个猜想：**    
mycat3（对齐缓冲）的平均耗时为775.7ms，而 mycat2的平均耗时为 779.1ms。虽然提升幅度不大，但表明对于 cat 操作，内存对齐可能带来微小的、可测量的性能改善。改善的原因可能是内核在处理对齐缓冲区时效率稍高，减少了内部数据整理开销。 

2.
`malloc`函数设计为通用的内存分配器，它的主要目标是：    
**内存利用率最大化**：malloc尽量减少内存碎片，通常只保证对齐到机器字长（如8字节或16字节）。 
**性能平衡**：页对齐需要额外的空间开销（每次分配可能浪费最多一页-1字节的空间），malloc选择了更通用的对齐策略。  
**标准兼容性**：C标准只要求malloc返回适合任何基本类型的对齐，不要求页对齐。     
**实现复杂性**：页对齐需要更复杂的内存管理算法，会增加malloc的实现复杂度和运行时开销。   

3. 

在我的实现中，我使用了以下策略：    
```c
char* align_alloc(size_t size)
{
    size_t page_size = io_blocksize();
    size_t total_size = size + page_size;
    
    void *raw_ptr = malloc(total_size);
    if (raw_ptr == NULL) {
        return NULL;
    }
    
    // 计算页对齐的地址
    uintptr_t aligned_addr = (raw_addr + page_size - 1) & ~(page_size - 1);
    char *aligned_ptr = (char*)aligned_addr;
    
    // 在对齐指针前存储原始指针
    *((void**)(aligned_ptr - sizeof(void*))) = raw_ptr;
    
    return aligned_ptr;
}

void align_free(void* ptr)
{
    if (ptr == NULL) return;
    
    // 从对齐指针前获取原始指针
    void *raw_ptr = *((void**)((char*)ptr - sizeof(void*)));
    free(raw_ptr);
}
```
**关键点**：    

分配额外空间确保有足够空间进行对齐  
在返回的对齐指针前8字节存储原始malloc返回的指针 
释放时从存储位置读取原始指针并传递给free    

这种方法的优点是简单可靠，缺点是需要额外的8字节存储空间。

## 任务4: 设置缓冲区大小为文件系统块大小的整数倍的cat

由于`cat`是涉及文件操作的，所以我们自然不能离开磁盘操作。我们在课内已经学到过，磁盘操作的基本单位是块。并且因为我们操作磁盘是经过了操作系统的一层抽象的，操作系统的文件系统也定义了一个操作文件的基本单位块，这个块的大小和磁盘的块的大小相关，但不总是相同。因此我们操作文件的时候实际接触到的块大小是文件系统的块大小。如果我们每次读取和写入文件的时候都按照文件系统的块大小来进行，也能提升性能。在这个任务中，你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat4.c`/`mycat4.rs`，可执行二进制文件命名为`mycat4`。
2. 修改你的函数`io_blocksize`，让你的缓冲区大小既考虑到内存页大小也考虑到文件系统的块大小。
3. 使用`hyperfine`测量`mycat4`的运行时间。
4. 保持系统编程的基本规范。

> 在完成这项任务的时候你需要注意以下几点：
> 1. 文件系统中的每个文件，块大小不总是相同的。
> 2. 有的文件系统可能会给出虚假的块大小，这种虚假的文件块大小可能根本不是2的整数次幂。

这个任务，你需要在文档中回答以下问题：
1. 为什么在设置缓冲区大小的时候需要考虑到文件系统块的大小的问题？
2. 对于上面提到的两个注意事项你是怎么解决的？

In [45]:
%%bash
export PATH="/root/.cargo/bin:$PATH"
hyperfine --warmup 3 './mycat4 test.txt'

Benchmark 1: ./mycat4 test.txt
  Time (mean ± σ):     848.6 ms ±  39.1 ms    [User: 109.3 ms, System: 738.2 ms]
  Range (min … max):   790.2 ms … 914.1 ms    10 runs
 


1. 
**I/O效率**：文件系统以块为单位管理数据。当读写操作的大小是块大小的整数倍时，文件系统可以：

- 避免读-修改-写操作
- 减少元数据更新频率
- 更好地利用磁盘缓存

**减少系统调用开销**：按块大小读写可以最大化每次系统调用传输的数据量，减少用户态-内核态切换开销。

**存储设备优化**：现代SSD和HDD都有最优的访问粒度，通常与文件系统块大小相关。

2. 
```c
size_t io_blocksize(int fd)
{
    struct stat st;
    if (fstat(fd, &st) == -1) {
        // 错误处理，使用默认值
        return default_page_size;
    }
    
    size_t fs_block_size = (size_t)st.st_blksize;
    // 针对具体文件获取块大小
}
```

**问题2：虚假的块大小处理**
```c
// 检查块大小是否合理
int is_power_of_two(size_t n) {
    return n > 0 && (n & (n - 1)) == 0;
}

if (!is_power_of_two(fs_block_size) || fs_block_size < 512) {
    fs_block_size = 4096; // 使用合理的默认值
}
```

**综合策略**：使用内存页大小和文件系统块大小的最小公倍数作为缓冲区大小，同时设置合理的上限避免过大的缓冲区。

## 任务5: 考虑系统调用开销情况下的cat

如果你正确完成了上面的任务，那么现在你的`cat`已经可以综合考虑内存页大小，内存页对齐和文件系统块大小的因素来设置缓冲区大小了。但是我们会发现，我们自己的`cat`性能仍然不如我们系统中的`cat`。并且如果你尝试过再进一步增大缓冲区的大小，你的`cat`性能还能更高。这是因为我们目前设置的缓冲区大小还不足以掩盖系统调用带来的开销。那么，我们的缓冲区究竟应该设置到什么大小才够呢？其实这个问题，我们仅仅使用理论分析是无法给出答案的，因为答案受到机器的硬件条件，操作系统的涉及等多重不确定因素的影响。但是，我们可以使用实验来解答这个问题。最后，我们还需要做出假设，不考虑上一个任务的注意事项1，也就是我们假设我们文件系统的大部分文件的块大小都一致（你可以使用我们的测试文件的块大小）。因此，设我们在之前的任务中设置的缓冲区大小是buf_size，我们需要通过实验找到一个倍数A，满足以下条件：
1. 当缓冲区大小小于A * buf_size的时候，文件的读写速度显著减小
2. 当缓冲区大小大于A * buf_size的时候，文件的读写速度并不显著提升
最终，我们就可以直接设置我们的`cat`中的缓冲区大小设置成buf_size的固定倍率。在这个任务中，你只少需要完成以下要求：
1. 编写一个实验脚本，尽量排除其他因素的影响，测量只在系统调用开销的影响下，你的系统最合适的缓冲区大小。并且给出这个大小下你的系统的读写速率。
2. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat5.c`/`mycat5.rs`，可执行二进制文件命名为`mycat5`。
3. 利用上面的实验结果，修改你的函数`io_blocksize`。
4. 使用`hyperfine`测量`mycat5`的运行时间。
5. 保持系统编程的基本规范。

> 提示：
> 1. `dd`命令可以用于复制一个文件(也就是读取并写入)，你可以使用命令行参数设置它的缓冲区大小，并且程序终止的时候可以报告平均文件读写速度。
> 2. Linux系统中存在`/dev`文件系统，这个目录下有很多特殊文件，其中有一些对你来说可能有用。`/dev/null`，你向这个文件写入的内容并不真正写入磁盘，并且不会对你的系统造成任何影响。`/dev/zero`，如果你尝试读取这个文件你会得到源源不断的`\0x0`，这个文件也并不真正的从磁盘中读取。

这个任务，你需要在文档中包括以下内容：
1. 解释一下你的实验脚本是怎么设计的。你应该尝试了多种倍率，请将它们的读写速率画成图表包含在文档中。

In [ ]:
#!/bin/bash

echo "Buffer Size (KB),output (GB/s)" > output.csv

# Base buffer size from mycat4 (e.g., page size or st_blksize, assume 4KB for this example)
base_buf_size_bytes=4096
# Let's try multipliers that give us common buffer sizes like
# 4K, 8K, 16K, 32K, 64K, 128K, 256K, 512K, 1024K
# Multipliers: 1, 2, 4, 8, 16, 32, 64, 128, 256
multipliers=(1 2 4 8 16 32 64 128 256 512) # Multipliers for base_buf_size
total_data_to_transfer="2G" # Total data to transfer for each test

# Calculate count for dd based on total data and current bs
# For simplicity in parsing dd output, we'll let dd calculate count if bs is given and size is specified for if/of.
# Or, more robustly, calculate count explicitly.
# Size in bytes for 2G = 2 * 1024 * 1024 * 1024
total_data_bytes=$((2 * 1024 * 1024 * 1024))

echo "Running dd tests with various buffer sizes..."

# ... (脚本前面部分不变) ...

for multiplier in "${multipliers[@]}"; do
    current_bs_bytes=$((base_buf_size_bytes * multiplier))
    current_bs_kb=$((current_bs_bytes / 1024))
    
    count=$(( (total_data_bytes + current_bs_bytes - 1) / current_bs_bytes ))

    echo "Testing with buffer size: ${current_bs_kb}KB (Multiplier: ${multiplier}x, bs=${current_bs_bytes}, count=${count})"

    # Run dd and capture stderr (where speed is reported)
    # Force LANG=C to get consistent dd output format if possible
    output=$(LANG=C dd if=/dev/zero of=/dev/null bs=${current_bs_bytes} count=${count} 2>&1 | grep 'bytes.*copied')
    
    if [ -n "$output" ]; then
        # More robust parsing:
        # Example output: "2147483648 bytes (2.1 GB, 2.0 GiB) copied, 0.123 s, 17.4 GB/s"
        # Or: "2147483648 bytes (2.1 GB, 2.0 GiB) copied, 1.234 s, 1740 MB/s"
        
        # Extract the part after the last comma, which contains "speed unit"
        speed_part=$(echo "$output" | awk -F', ' '{print $NF}') # $NF should be "17.4 GB/s" or "1740 MB/s"
        
        speed_value=$(echo "$speed_part" | awk '{print $1}') # "17.4" or "1740"
        speed_unit=$(echo "$speed_part" | awk '{print $2}')  # "GB/s" or "MB/s"

        output_gb_s=""

        if [[ "$speed_unit" == "GB/s" ]]; then
            output_gb_s=$speed_value
        elif [[ "$speed_unit" == "MB/s" ]]; then
            # Convert MB/s to GB/s using bc for floating point arithmetic
            output_gb_s=$(echo "scale=3; $speed_value / 1024" | bc)
        elif [[ "$speed_unit" == "kB/s" || "$speed_unit" == "KB/s" ]]; then # Less likely for /dev/zero to /dev/null
            output_gb_s=$(echo "scale=6; $speed_value / 1024 / 1024" | bc)
        else
            echo "Warning: Unknown speed unit '$speed_unit' from dd output: $output"
            output_gb_s="ERROR" # Mark as error
        fi
        
        if [[ "$output_gb_s" != "ERROR" ]]; then
            echo "${current_bs_kb},${output_gb_s}" >> dd_output.csv
            echo "output: ${output_gb_s} GB/s (Original: $speed_part)"
        else
            echo "${current_bs_kb},ERROR" >> dd_output.csv
            echo "Failed to parse output from dd output."
        fi
    else
        echo "Failed to get dd output for bs=${current_bs_kb}KB"
        echo "${current_bs_kb},ERROR" >> dd_output.csv
    fi
    sleep 1 # Small pause between tests
done

# ... (脚本后面 Python 部分不变) ...
echo "Experiment finished. Results in dd_output.csv"
echo "Now plotting the results..."

# Python script to plot the results
python3 - <<END_PYTHON_SCRIPT
import matplotlib.pyplot as plt
import pandas as pd

try:
    data = pd.read_csv('dd_output.csv')
    # Ensure output is numeric, coercing errors to NaN and dropping them
    data['output (GB/s)'] = pd.to_numeric(data['output (GB/s)'], errors='coerce')
    data.dropna(subset=['output (GB/s)'], inplace=True)

    if not data.empty:
        plt.figure(figsize=(10, 6))
        plt.plot(data['Buffer Size (KB)'], data['output (GB/s)'], marker='o')
        plt.title('dd output vs. Buffer Size (Input: /dev/zero, Output: /dev/null)')
        plt.xlabel('Buffer Size (KB)')
        plt.ylabel('output (GB/s)')
        plt.xscale('log', base=2) # Use log scale for buffer size for better visualization
        plt.xticks(data['Buffer Size (KB)'], data['Buffer Size (KB)'].astype(str)) # Show all x-ticks
        plt.grid(True, which="both", ls="--")
        plt.savefig('dd_output_plot.png')
        print("Plot saved as dd_output_plot.png")
        # plt.show() # Uncomment to display plot if running in a GUI environment
    else:
        print("No valid data to plot.")
except FileNotFoundError:
    print("Error: dd_output.csv not found. Run the bash script first.")
except Exception as e:
    print(f"An error occurred during plotting: {e}")

END_PYTHON_SCRIPT

In [ ]:
%%bash
hyperfine --warmup 3 './mycat5 test.txt'

Benchmark 1: ./mycat5 test.txt
  Time (mean ± σ):     436.0 ms ±  25.9 ms    [User: 4.6 ms, System: 429.8 ms]
  Range (min … max):   400.4 ms … 474.3 ms    10 runs
 


我设计的实验脚本使用`dd`命令测试不同缓冲区大小的性能：

**实验设计原理**：
- 使用`/dev/zero`作为输入源，避免磁盘读取瓶颈
- 使用`/dev/null`作为输出目标，避免磁盘写入瓶颈
- 测试多个缓冲区大小倍数（1x, 2x, 4x, 8x, 16x, 32x, 64x, 128x, 256x页面大小）
- 每个测试使用相同的数据量（2GB）

**关键控制变量**：
- 固定测试文件大小
- 固定测试环境（同一台机器、同一时间段）
- 使用特殊设备文件排除磁盘I/O影响

**预期结果分析**：
- 小缓冲区：系统调用频率高，总体性能差
- 中等缓冲区：系统调用开销与内存拷贝开销平衡，性能最优
- 大缓冲区：系统调用开销被摊薄，但内存拷贝和缓存污染增加

![](/root/meowhw-Eraeus326/dd_output_plot.png)


大概512KB时性能最好

## 任务6: 使用了系统调用`fdadvice`的cat

虽然几乎我们的这个lab都在讨论设置缓冲区大小的问题，但是实际上我们的系统中的`cat`还在其他很多方面做了不少优化。这些优化在多数时候和缓冲区相比都不起决定性作用，但是我们也可以从中学习到不少有趣的技巧。这里我们就只尝试其中一个，使用系统调用`fadvise`。这个系统调用可以提示文件系统我们将会以什么样的模式来读写文件，这样操作系统可以设置合适的readahead窗口为文件的读写做准备。在这个任务中，你需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat6.c`/`mycat6.rs`，可执行二进制文件命名为`mycat6`。
2. 在你的代码中使用`fadvise`进行优化。
3. 使用`hyperfine`测量`mycat6`的运行时间。
4. 保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 你是如何设置`fadvise`的参数的？
2. 对于顺序读写的情况，文件系统可以如何调整readahead？对于随机读写的情况呢？

In [53]:
%%bash
hyperfine --warmup 3 './mycat6 test.txt'

Benchmark 1: ./mycat6 test.txt
  Time (mean ± σ):     437.2 ms ±  12.1 ms    [User: 0.7 ms, System: 436.7 ms]
  Range (min … max):   421.5 ms … 461.1 ms    10 runs
 


我使用了以下`fadvise`设置：

```c

posix_fadvise(fd, 0, 0, POSIX_FADV_SEQUENTIAL);


2. 

**顺序读写优化**：
- **增大readahead窗口**：内核会预读更多连续的块到页缓存
- **预取策略**：基于访问模式预测，提前加载可能需要的数据
- **I/O调度优化**：将连续的读请求合并，减少磁盘寻道时间
- **缓存策略**：优先保留顺序访问的数据在缓存中

**随机读写优化**：
- **减小readahead窗口**：避免无用的预读，减少I/O浪费
- **缓存策略调整**：采用LRU等算法，快速淘汰不太可能再次访问的数据
- **I/O优先级**：随机访问通常优先级较低，避免影响顺序I/O
- **预取禁用**：对于明确的随机访问模式，完全禁用预取机制


## 任务7: 总结

经过了上面的所有任务，我们已经成功探索我们系统中最常用的工具`cat`所使用的各种优化。我相信你对涉及系统IO的编程也有了更深刻的理解。现在请你整理汇总上面每个任务你所写的`mycatx`以及系统的`cat`的测量数据，使用一个柱状图来展示。并且请你分析一下你得到的结果：它们符合你的预期吗？为什么？这个结果给你带来了什么启示？

这个任务，你需要在文档中包含以下内容：
1. 你的全部实验结果的柱状图。
2. 你对上述实验结果的分析。

In [ ]:
# 这里填入你用于画图的python代码

import matplotlib.pyplot as plt
import numpy as np

# --- REPLACE WITH YOUR ACTUAL MEASURED MEAN TIMES IN MILLISECONDS ---
programs = ['System cat', 'mycat1', 'mycat2', 'mycat3', 'mycat4', 'mycat5', 'mycat6']
# Times in milliseconds for better scale on y-axis if mycat1 is included
times_ms = [
    456,      # System cat
    1500000,  # mycat1 (char-by-char)
    779,      # mycat2 (page-size buffer)
    775,      # mycat3 (aligned page-size buffer)
    848,      # mycat4 (fs_block_size aware, aligned)
    436,      # mycat5 (256KB buffer, aligned) - EXAMPLE, use your value
    436       # mycat6 (256KB buffer, aligned, fadvise) - EXAMPLE, use your value
]
# --- END OF DATA TO REPLACE ---

# If mycat1's time is excessively large, it will dwarf other bars.
# Consider plotting two charts: one with all, one without mycat1 for better detail.

# Plot 1: All programs
plt.figure(figsize=(12, 7))
bars = plt.bar(programs, times_ms, color=['skyblue', 'salmon', 'lightgreen', 'gold', 'lightcoral', 'mediumpurple', 'lightseagreen'])
plt.ylabel('Mean Execution Time (ms)')
plt.title('Performance Comparison of cat Implementations (All)')
plt.xticks(rotation=45, ha="right")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add text labels on bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.05 * max(times_ms), f'{yval:.0f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('mycat_performance_all.png')
print("Plot 'mycat_performance_all.png' saved.")
# plt.show()

# Plot 2: Excluding mycat1 for better detail on optimized versions
programs_optimized = programs[0:1] + programs[2:] # System cat + mycat2 onwards
times_ms_optimized = times_ms[0:1] + times_ms[2:]

plt.figure(figsize=(10, 6))
bars_optimized = plt.bar(programs_optimized, times_ms_optimized, color=['skyblue', 'lightgreen', 'gold', 'lightcoral', 'mediumpurple', 'lightseagreen'])
plt.ylabel('Mean Execution Time (ms)')
plt.title('Performance Comparison of cat Implementations (Optimized Versions)')
plt.xticks(rotation=45, ha="right")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add text labels on bars for the optimized plot
for bar in bars_optimized:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.02 * max(times_ms_optimized), f'{yval:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('mycat_performance_optimized.png')
print("Plot 'mycat_performance_optimized.png' saved.")
# plt.show()


![](/root/meowhw-whisperLyx-1/work_my/mycat_performance_optimized.png)

预期的性能提升趋势：
1. **mycat1 → mycat2**：显著提升（10-100倍），缓冲区消除了大量系统调用
2. **mycat2 → mycat3**：中等提升（1-2%），页对齐减少内存访问开销
3. **mycat3 → mycat4**：本来应该是提升的，但是反而性能降低了，可能是没有选取好参考文件系统块的大小。
4. **mycat4 → mycat5**：显著提升，寻找到的最优缓冲区进一步减少系统调用
5. **mycat5 → mycat6**：基本没有提升，可能是我在fadvise函数调用上优化不够。

##### 实验启示：    

1. **缓冲区的重要性**：适当的缓冲区设计是高性能I/O的关键
2. **内存对齐的作用**：页对齐可以显著提升内存访问效率
3. **文件系统特性**：了解并利用文件系统特性可以优化I/O性能
4. **系统调用开销**：大缓冲区可以摊薄系统调用的固定开销
5. **内核提示机制**：fadvise等系统调用可以帮助内核做出更好的决策


